In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler

class TrafficFlowCNN(nn.Module):
    def __init__(self, input_channels=3, hidden_channels=2, output_channels=1, num_nodes=307, time_steps=12):
        super(TrafficFlowCNN, self).__init__()
        
        # First Conv2D layer (input: 3 channels -> output: 2 channels)
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=hidden_channels, kernel_size=(3, 2), padding=(1, 0))
        self.relu = nn.ReLU()
        
        # Second Conv2D layer (input: 2 channels -> output: 1 channel)
        self.conv2 = nn.Conv2d(in_channels=hidden_channels, out_channels=output_channels, kernel_size=(2, 1), padding=(0, 0))
        
    def forward(self, C):
        # Extract A (Traffic Flow Data) from C (assuming it is the first channel)
        A = C[:, :1, :, :]  # First feature is traffic flow
        
        # Apply two convolutional layers
        out = self.conv1(C)
        out = self.relu(out)
        out = self.conv2(out)
        
        # Residual connection: Add A to the final output
        X = out + A
        return X

# Load Data
file_path = 'C:/Users/Hoda/A - Uni/thesis/AGCRN_GSL-init/data/PEMS04/pems04.npz'
data = np.load(file_path)['data']  # Shape: (num_samples, num_nodes, time_steps, features)

# Normalize data using StandardScaler
data = data.astype(np.float32)
scaler = StandardScaler()
num_samples, num_nodes, features = data.shape

data_reshaped = data.reshape(num_samples * num_nodes, features)
data_normalized = scaler.fit_transform(data_reshaped).reshape(num_samples, num_nodes, features)

# Convert to PyTorch tensor
traffic_tensor = torch.tensor(data_normalized, dtype=torch.float32).permute(0, 2, 1).unsqueeze(-1)

# Model Initialization
batch_size = 64
input_dim = 3
time_steps = 12  # Define time_steps
model = TrafficFlowCNN(input_channels=input_dim, num_nodes=num_nodes, time_steps=time_steps)

# Example batch extraction
input_batch = traffic_tensor[:batch_size].permute(0, 1, 3, 2)  # Shape: (batch_size, 3, num_nodes, time_steps)

# Forward pass
output = model(input_batch)
print("Output shape:", output.shape)  # Expected: (batch_size, 1, num_nodes, time_steps)


RuntimeError: Calculated padded input size per channel: (1 x 306). Kernel size: (2 x 1). Kernel size can't be greater than actual input size

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ThreePhaseFeatureExtractor(nn.Module):
    def __init__(self, input_channels=3, output_channels=1):
        """
        Multi-modal traffic feature extractor with two convolutional layers + residual connection.
        """
        super(ThreePhaseFeatureExtractor, self).__init__()

        # First convolutional layer (3×2 Conv2D): Converts 3 channels to 2 channels
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=2, kernel_size=(3,2), padding=(1,0))

        # Second convolutional layer (2×1 Conv2D): Converts 2 channels to 1 channel
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=output_channels, kernel_size=(2,1), padding=(0,0))

    def forward(self, C):
        """
        Args:
            C (torch.Tensor): Input traffic data (B, T, N, 3) -> (batch_size, time_steps, num_nodes, features)

        Returns:
            torch.Tensor: Processed traffic representation (B, T, N, 1)
        """
        B, T, N, F = C.shape  # Extract shape

        # Reshape to match Conv2D input format: (batch, channels, nodes, time_steps)
        C = C.permute(0, 3, 2, 1)  # New shape: (B, 3, N, T)

        # First Convolutional Layer
        C1 = F.relu(self.conv1(C))  # Output shape: (B, 2, N, T)

        # Second Convolutional Layer
        C2 = F.relu(self.conv2(C1))  # Output shape: (B, 1, N, T)

        # Residual Connection: Add traffic flow (first channel) to processed features
        A = C[:, 0:1, :, :]  # Extract only the traffic flow data
        X = A + C2  # Combine extracted traffic flow with convolutional features

        # Reshape back to AGCRN input format: (B, T, N, 1)
        X = X.permute(0, 3, 2, 1)

        return X
